https://heartbeat.comet.ml/how-to-build-a-text-classification-model-using-huggingface-transformers-and-comet-4d40236e8f84



In [2]:
import pandas as pd
import numpy as np

In [3]:
!pip install huggingface

  Using cached huggingface-0.0.1-py3-none-any.whl (2.5 kB)


In [4]:
!pip install huggingface transformers

In [5]:
df = pd.read_csv('filtered_data.csv')

/var/folders/b1/vw0sn5w90nnccbfn7bskv0jm0000gn/T/ipykernel_27870/2619961329.py:1: DtypeWarning: Columns (0,1,10,11,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('filtered_data.csv')


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911730 entries, 0 to 911729
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              911730 non-null  object 
 1   tweet_id             911716 non-null  object 
 2   tweet_text           911716 non-null  object 
 3   place_id             891950 non-null  object 
 4   place_name           891950 non-null  object 
 5   full_place_name      891950 non-null  object 
 6   country              891950 non-null  object 
 7   country_code         891950 non-null  object 
 8   place_type           891950 non-null  object 
 9   username             891950 non-null  object 
 10  profile_description  791620 non-null  object 
 11  profile_name         891943 non-null  object 
 12  profile_location     768326 non-null  object 
 13  lat                  891950 non-null  object 
 14  lon                  859170 non-null  float64
 15  clean_text       

In [7]:
df = df.drop(['withheld'], axis = 1)

KeyError: "['withheld'] not found in axis"

In [8]:
country_list = ['United States', 'United Kingdom', 'Canada', 'India', 'Australia', 'Nigeria']
df= df[df['country'].isin(country_list)]

In [9]:
df['country'].value_counts()

United States     668904
United Kingdom     73613
Canada             41151
India              29255
Australia          24717
Nigeria            21530
Name: country, dtype: int64

In [10]:
df.head()

,user_id,tweet_id,tweet_text,place_id,place_name,full_place_name,country,country_code,place_type,username,profile_description,profile_name,profile_location,lat,lon,clean_text
0,1051074201082388480,1637850668214960128,@Gajendr70729189 @amitsharma2704 @1shankarshar...,5f55bb82cf16ac81,Bengaluru South,"Bengaluru South, India",India,IN,city,Pavanasoonu,Global SAP Tech. #Jyotishkatti. Spiritual cou...,Pavanasoonu,"Bengaluru, India",77.496999,12.923098,Including my SAP technology business. Thank ...
1,1051074201082388480,1637818231863001090,@JatinPandyaVBNJ @1shankarsharma Indeed. Nadi...,5f55bb82cf16ac81,Bengaluru South,"Bengaluru South, India",India,IN,city,Pavanasoonu,Global SAP Tech. #Jyotishkatti. Spiritual cou...,Pavanasoonu,"Bengaluru, India",77.496999,12.923098,Indeed. Nadi Jyotish has excellent rules. N...
2,1051074201082388480,1637737394911969280,@1shankarsharma Namaste. How many years have ...,5f55bb82cf16ac81,Bengaluru South,"Bengaluru South, India",India,IN,city,Pavanasoonu,Global SAP Tech. #Jyotishkatti. Spiritual cou...,Pavanasoonu,"Bengaluru, India",77.496999,12.923098,Namaste. How many years have you spent learn...
3,1051074201082388480,1637733079002537986,An excellent example of jumping into conclusio...,5f55bb82cf16ac81,Bengaluru South,"Bengaluru South, India",India,IN,city,Pavanasoonu,Global SAP Tech. #Jyotishkatti. Spiritual cou...,Pavanasoonu,"Bengaluru, India",77.496999,12.923098,An excellent example of jumping into conclusio...
4,1051074201082388480,1637660117624836096,The want for financial security must be balanc...,5f55bb82cf16ac81,Bengaluru South,"Bengaluru South, India",India,IN,city,Pavanasoonu,Global SAP Tech. #Jyotishkatti. Spiritual cou...,Pavanasoonu,"Bengaluru, India",77.496999,12.923098,The want for financial security must be balanc...


In [23]:
filtered_df = df[['user_id', 'tweet_id', 'tweet_text', 'country']]

In [12]:
filtered_df.head()

,user_id,tweet_id,tweet_text,country
0,1051074201082388480,1637850668214960128,@Gajendr70729189 @amitsharma2704 @1shankarshar...,India
1,1051074201082388480,1637818231863001090,@JatinPandyaVBNJ @1shankarsharma Indeed. Nadi...,India
2,1051074201082388480,1637737394911969280,@1shankarsharma Namaste. How many years have ...,India
3,1051074201082388480,1637733079002537986,An excellent example of jumping into conclusio...,India
4,1051074201082388480,1637660117624836096,The want for financial security must be balanc...,India


In [24]:
updated_df = filtered_df.astype({'user_id':int, 'tweet_id': int, 'tweet_text': str, 'country':str})

In [14]:
updated_df.to_csv('updated_df.csv')

In [15]:
updated_df.dtypes

user_id        int64
tweet_id       int64
tweet_text    object
country       object
dtype: object

In [16]:
updated_df.shape

(859170, 4)

In [17]:
id2label = {0: "United States", 1: "United Kingdom", 2: "Canada", 3: "Australia", 4: "India", 5: "Nigeria"}
label2id = {"United States": 0, "United Kingdom": 1, "Canada": 2, "Australia": 3, "India": 4, "Nigeria": 5}

In [18]:
sample_df = updated_df.sample(5000, random_state = 1)

In [19]:
sample_df.head()

,user_id,tweet_id,tweet_text,country
778097,490104241,1637289107557879808,My aunt married a white man with the last name...,United States
704769,2486046872,1635484083256979456,@GovRonDeSantis It shouldn't include theologic...,United States
603215,1374446305146138626,1633236299640365057,"I had my best day ever at work, today!!!!! Fou...",United States
455462,100339412,1637856072521363456,@DawiLDanger My earbuds are from Big Lots (a c...,United States
626596,1488755185,1637782944667320320,@CraigAFountain Same! Thanks for having me!!!,United States


In [25]:
sample_df['country'] = sample_df['country'].apply(lambda x: label2id[x])

KeyError: 0

In [ ]:
sample_df.head()

,user_id,tweet_id,tweet_text,country
810865,490104241,1637289107557879808,My aunt married a white man with the last name...,0
737537,2486046872,1635484083256979456,@GovRonDeSantis It shouldn't include theologic...,0
635983,1374446305146138626,1633236299640365057,"I had my best day ever at work, today!!!!! Fou...",0
488230,100339412,1637856072521363456,@DawiLDanger My earbuds are from Big Lots (a c...,0
659364,1488755185,1637782944667320320,@CraigAFountain Same! Thanks for having me!!!,0


In [26]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 778097 to 705133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     5000 non-null   int64 
 1   tweet_id    5000 non-null   int64 
 2   tweet_text  5000 non-null   object
 3   country     5000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 195.3+ KB


In [ ]:
!pip install datasets

In [27]:
from datasets import Dataset

/Users/stephentoner/miniconda3/envs/SI699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
train, validate, test = np.split(sample_df.sample(frac=1), [int(.6*len(sample_df)), int(.8*len(sample_df))])

In [29]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index = False)
validate.to_csv('validate.csv', index = False)

In [30]:
ds_train = Dataset.from_csv('train.csv')
ds_val = Dataset.from_csv('validate.csv')
ds_test = Dataset.from_csv('test.csv')

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 269.25it/s]


Dataset csv downloaded and prepared to /Users/stephentoner/.cache/huggingface/datasets/csv/default-eebe832ecc69a198/0.0.0. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 589.25it/s]


Dataset csv downloaded and prepared to /Users/stephentoner/.cache/huggingface/datasets/csv/default-a66aa153b2d656dd/0.0.0. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 529.92it/s]
                                                        

Dataset csv downloaded and prepared to /Users/stephentoner/.cache/huggingface/datasets/csv/default-a295528222c270bc/0.0.0. Subsequent calls will reuse this data.


In [31]:
ds = {"train": ds_train, "validation": ds_val, "test": ds_test}

In [32]:
ds_train[0]

{'user_id': 42751738,
 'tweet_id': 1637119228338753538,
 'tweet_text': '@DanWolken 298 vs 5 in kenpom.',
 'country': 0}

In [33]:
ds_train

Dataset({
    features: ['user_id', 'tweet_id', 'tweet_text', 'country'],
    num_rows: 3000
})

In [34]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("Twitter/twhin-bert-base", num_labels=6, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at Twitter/twhin-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-ba

In [35]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, DataCollatorWithPadding
tokenizer = AutoTokenizer.from_pretrained('Twitter/twhin-bert-base')

In [36]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def preprocess_function(examples):
    label = examples["country"] 
    # examples['tweet_text'].apply(preprocess)
    examples = tokenizer(preprocess(examples["tweet_text"]), truncation=True, padding="max_length", max_length=256, return_tensors='pt')
    for key in examples:
        examples[key] = examples[key].squeeze(0)
    examples["label"] = label
    return examples

for split in ds:
    ds[split] = ds[split].map(preprocess_function, remove_columns=['user_id', 'tweet_id', 'tweet_text', 'country'])
    ds[split].set_format('pt')

In [37]:
example = next(iter(ds['test']))

In [39]:
tokenizer.decode(example['input_ids'])

'<s> @user @user @user @user @user @user @user @user @user @user @user Sure we will come over. Which country do live in? 🙋 ♀️💜<unk> <unk>🐾🇦🇺</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [40]:
ds['train']["label"][:10]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
!pip install evaluate

In [41]:
import evaluate

accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels), "f1":f1_metric.compute(predictions=predictions, references=labels, average="weighted")}

In [ ]:
import torch
class TwitterTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = torch.nn.functional.cross_entropy(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = TwitterTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.827472,{'accuracy': 0.811},{'f1': 0.7908860889089976}
2,No log,0.743486,{'accuracy': 0.829},{'f1': 0.8021886040993591}


Trainer is attempting to log a value of "{'accuracy': 0.811}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7908860889089976}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.829}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8021886040993591}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=376, training_loss=0.33468847071870844, metrics={'train_runtime': 370.3007, 'train_samples_per_second': 16.203, 'train_steps_per_second': 1.015, 'total_flos': 789361514496000.0, 'train_loss': 0.33468847071870844, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "{'accuracy': 0.829}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8021886040993591}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.7434858679771423,
 'eval_accuracy': {'accuracy': 0.829},
 'eval_f1': {'f1': 0.8021886040993591},
 'eval_runtime': 17.3352,
 'eval_samples_per_second': 57.686,
 'eval_steps_per_second': 3.634,
 'epoch': 2.0}

In [ ]:
predictions = trainer.predict(ds['test'])

In [ ]:
predictions

PredictionOutput(predictions=array([[-1.1753128e-01, -1.6444532e-02, -5.3166467e-01, -3.4069392e-01,
         2.9098943e-01,  8.0153126e-01],
       [-6.2814379e-01, -8.7985867e-01, -3.4760940e-01,  4.1945424e-02,
         2.1285038e+00, -1.0165071e+00],
       [ 5.3725195e+00,  5.8511466e-02, -1.9380964e-01, -8.1769091e-01,
        -2.7476633e+00, -1.7396835e+00],
       ...,
       [-2.5696127e+00,  1.2272263e+00, -3.3492857e-01,  3.3991376e-01,
         2.7463822e+00, -7.9077470e-01],
       [ 5.1196032e+00,  4.8865637e-01,  2.2010575e-01, -9.5231241e-01,
        -3.2083671e+00, -1.4377913e+00],
       [ 5.5168905e+00,  1.3789328e-02,  4.0565976e-03, -1.0782397e+00,
        -2.8317249e+00, -1.6308876e+00]], dtype=float32), label_ids=array([0, 4, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1,
       5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 5, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0

In [ ]:
predictions[0]

array([[-1.1753128e-01, -1.6444532e-02, -5.3166467e-01, -3.4069392e-01,
         2.9098943e-01,  8.0153126e-01],
       [-6.2814379e-01, -8.7985867e-01, -3.4760940e-01,  4.1945424e-02,
         2.1285038e+00, -1.0165071e+00],
       [ 5.3725195e+00,  5.8511466e-02, -1.9380964e-01, -8.1769091e-01,
        -2.7476633e+00, -1.7396835e+00],
       ...,
       [-2.5696127e+00,  1.2272263e+00, -3.3492857e-01,  3.3991376e-01,
         2.7463822e+00, -7.9077470e-01],
       [ 5.1196032e+00,  4.8865637e-01,  2.2010575e-01, -9.5231241e-01,
        -3.2083671e+00, -1.4377913e+00],
       [ 5.5168905e+00,  1.3789328e-02,  4.0565976e-03, -1.0782397e+00,
        -2.8317249e+00, -1.6308876e+00]], dtype=float32)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
predictions_array = np.argmax(predictions[0], axis = 1)

In [ ]:
predictions_array

array([5, 4, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       5, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4,
       0, 0, 0, 0, 1, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,

In [ ]:
labels_array = predictions[1]
labels_array

array([0, 4, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1,
       5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 5, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 3, 2, 0, 0, 0, 0, 0, 0, 2, 2, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4,
       0, 0, 0, 0, 3, 0, 0, 0, 4, 0, 3, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1,
       0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 4, 0, 1, 0, 1, 0, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0,

In [ ]:
confusion_matrix(labels_array, predictions_array)

array([[785,  11,   2,   0,   3,   3],
       [ 45,  30,   2,   1,   1,   0],
       [ 31,   0,   7,   1,   0,   0],
       [ 16,   4,   2,   2,   1,   1],
       [ 11,   2,   0,   2,  23,   0],
       [  5,   0,   0,   0,   2,   7]])